<a href="https://colab.research.google.com/github/yhb1834/cv_prj2/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Detection을 위한 Tensorflow 모듈 설치
import tensorflow as tf
import tensorflow_hub as hub

# 이미지를 위한 모듈
import matplotlib.pyplot as plt #차트 그리기
import tempfile # 임시 파일/디렉터리 생성 
from six.moves.urllib.request import urlopen # url처리 (six: python2, 3 동시처리 가능한 모듈)
from six import BytesIO #데이터를 BytesIO로 읽은 뒤 이미지 Open

# For drawing onto the image. 배열
import numpy as np

# 파이썬 이미지 처리 pkg
from PIL import Image # 이미지 생성, 불러오기 등 제공
from PIL import ImageColor # color table 제공
from PIL import ImageDraw # 이미지에 텍스트 쓰기
from PIL import ImageFont # 이미지 내 폰트 설정
from PIL import ImageOps # 이미지 변형 

import time # 시간측정을 위한 함수

# Tensorflow 버전 출력 및 GPU 체크
print("Tensorflow version:", tf.__version__)
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

Tensorflow version: 2.8.2
The following GPU devices are available: 
